In [1]:
import pandas as pd
import numpy as np
import os

In [8]:
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

In [ ]:
np.random.seed(42)

n = 1000

data = pd.DataFrame({

    'age': np.random.randint(18, 60, n),
    'days_since_last_login': np.random.randint(0, 100, n),
    'login_frequency': np.random.randint(1, 50, n),
    'total_time_spent': np.random.randint(10, 500, n),
    'avg_session_time': np.random.uniform(5, 60, n),
    'courses_enrolled': np.random.randint(1, 10, n),
    'assignments_completed': np.random.randint(0, 20, n),
    'videos_watched': np.random.randint(1, 300, n),
    'forum_posts': np.random.randint(0, 50, n),
    'subscription_length': np.random.randint(1, 365, n),
    'preferred_time': np.random.choice(['morning', 'evening', 'night'], n),
    'student_type': np.random.choice(['active', 'passive'], n),
    'country': np.random.choice(['France','Germany','Spain','Italy'], n),
    'dropout': np.random.choice([0,1], n)
})

# Ajouter des valeurs manquantes
data.loc[np.random.choice(data.index, 100), 'age'] = np.nan

# Ajouter des valeurs aberrantes
data.loc[np.random.choice(data.index, 50), 'total_time_spent'] = -999

data.to_csv('../data/raw/student_data.csv', index=False)

data.head()

In [10]:
df = pd.read_csv('../data/raw/student_data.csv')

In [11]:
df.head()
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          0 non-null      object
 1   study_hours  0 non-null      object
 2   attendance   0 non-null      object
 3   dropout      0 non-null      object
dtypes: object(4)
memory usage: 132.0+ bytes


,0
age,0
study_hours,0
attendance,0
dropout,0


In [14]:
# 1. Compter les valeurs manquantes
print("AVANT NETTOYAGE:")
print(df.isnull().sum())
print(f"Valeurs négatives dans total_time_spent: {(df['total_time_spent'] < 0).sum()}")


AVANT NETTOYAGE:
age            0
study_hours    0
attendance     0
dropout        0
dtype: int64


KeyError: 'total_time_spent'

In [6]:
# 2. Créer une copie pour le nettoyage
df_clean = df.copy()


In [9]:
# 3. Traiter les valeurs aberrantes de total_time_spent
# Remplacer -999 par NaN (pour les traiter comme manquantes)
df_clean.loc[df_clean['total_time_spent'] < 0, 'total_time_spent'] = np.nan
print(f"\nAprès remplacement des négatifs: {df_clean['total_time_spent'].isnull().sum()} NaN dans total_time_spent")




Après remplacement des négatifs: 47 NaN dans total_time_spent


In [10]:
# 4. Imputer les valeurs manquantes
# Pour age : remplacer par la médiane (moins sensible aux outliers que la moyenne)
median_age = df_clean['age'].median()
df_clean['age'].fillna(median_age, inplace=True)

# Pour total_time_spent : remplacer par la moyenne
mean_time = df_clean['total_time_spent'].mean()
df_clean['total_time_spent'].fillna(mean_time, inplace=True)



/tmp/ipython-input-1787202007.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['age'].fillna(median_age, inplace=True)
/tmp/ipython-input-1787202007.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [11]:
# 5. Vérifier qu'il n'y a plus de valeurs manquantes
print("\nAPRÈS NETTOYAGE:")
print(df_clean.isnull().sum())
print(f"Min total_time_spent: {df_clean['total_time_spent'].min()}")


APRÈS NETTOYAGE:
age                      0
days_since_last_login    0
login_frequency          0
total_time_spent         0
avg_session_time         0
courses_enrolled         0
assignments_completed    0
videos_watched           0
forum_posts              0
subscription_length      0
preferred_time           0
student_type             0
country                  0
dropout                  0
dtype: int64
Min total_time_spent: 10.0


In [14]:
# ============================================
# ENCODAGE DES VARIABLES CATÉGORIELLES
# ============================================

print("="*50)
print("ENCODAGE DES DONNÉES")
print("="*50)


ENCODAGE DES DONNÉES


In [15]:
# 1. Identifier les types de variables
print("\n🔍 Variables catégorielles dans nos données:")
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    print(f"   - {col}: {df_clean[col].unique().tolist()}")




🔍 Variables catégorielles dans nos données:
   - preferred_time: ['evening', 'night', 'morning']
   - student_type: ['active', 'passive']
   - country: ['Germany', 'Spain', 'Italy', 'France']


In [16]:
# 2. One-Hot Encoding (pour variables sans ordre)
print("\n✅ Application de One-Hot Encoding...")
df_encoded = pd.get_dummies(
    df_clean, 
    columns=['preferred_time', 'student_type', 'country'],  # variables à encoder
    drop_first=True,  # pour éviter la redondance
    dtype=int
)




✅ Application de One-Hot Encoding...


In [17]:
# 3. Vérification
print(f"\n📊 Shape avant encodage: {df_clean.shape}")
print(f"📊 Shape après encodage: {df_encoded.shape}")
print(f"\n📋 Nouvelles colonnes créées:")
new_cols = set(df_encoded.columns) - set(df_clean.columns)
for col in sorted(new_cols):
    print(f"   - {col}")




📊 Shape avant encodage: (1000, 14)
📊 Shape après encodage: (1000, 17)

📋 Nouvelles colonnes créées:
   - country_Germany
   - country_Italy
   - country_Spain
   - preferred_time_morning
   - preferred_time_night
   - student_type_passive


In [18]:
# 4. Aperçu du résultat
print("\n👀 Aperçu des données encodées (5 premières lignes):")
print(df_encoded.head())




👀 Aperçu des données encodées (5 premières lignes):
    age  days_since_last_login  login_frequency  total_time_spent  \
0  56.0                     28               49         107.00000   
1  46.0                     46               49          74.00000   
2  32.0                     67               44         256.87723   
3  25.0                     75               28         350.00000   
4  38.0                     44               47          14.00000   

   avg_session_time  courses_enrolled  assignments_completed  videos_watched  \
0         35.193925                 8                      1             266   
1         17.060068                 4                     17              95   
2          6.851805                 5                      8             262   
3         16.804544                 2                      5              71   
4         27.886679                 9                     10             219   

   forum_posts  subscription_length  dropout  prefe

In [22]:
# 5. Sauvegarde
df_encoded.to_csv('../data/processed/student_data_encoded.csv', index=False)
print("\n💾 Données encodées sauvegardées dans data/processed/")


💾 Données encodées sauvegardées dans data/processed/
